In [46]:

using ITensors
using ITensors: position!
using KrylovKit
using Printf


# Simple TDVP

Below is a code for a simple TDVP implementation (both for 1-site & 2-site variant). This is based on the ITensorTDVP. The function tdvp! takes the folllowing arguments.

**Initial wavefunction** <br>
**Hamiltonian :** (in MPO form) to time evolve the wave function<br>
**dt(time step)** <br>
**tf(final time )** : time upto which you want to evolve the system)<br>
**reverse_step_val :** Decides whether you need the right-to-left swap (Changes the accuracy O(dt^3) for true,O(dt^2) for no)

Time is entered as real for time evolution. The i is multiplied internally.

In [47]:
struct TDVP end
set_nsite!(PH::ProjMPO,nsite) = (PH.nsite = nsite)

function solver(H, t, psi0; kwargs...)
    solver_kwargs = (;
      ishermitian=get(kwargs, :ishermitian, true),
      issymmetric=get(kwargs, :issymmetric, true),
      tol=get(kwargs, :solver_tol, 1E-12),
      krylovdim=get(kwargs, :solver_krylovdim, 30),
      maxiter=get(kwargs, :solver_maxiter, 100),
      verbosity=get(kwargs, :solver_outputlevel, 0),
      eager=true,
    )
    psi, info = exponentiate(H, t, psi0; solver_kwargs...)
    return psi, info
end

function measure_Sz(psi,n)
    psi = ITensors.orthogonalize(psi,n)
    sn = siteind(psi,n)
    Sz = scalar(dag(prime(psi[n],"Site"))*op("Sz",sn)*psi[n])
    return real(Sz)
end


function tdvp!(ψ::MPS,H::MPO,dt::Float64,tf::Float64,nsite::Int64,reverse_step_val; kwargs...)
    
    num_time_steps = Int(tf/dt)
    
    #kwargs
   
    #The error tolerance for `KrylovKit.exponentiate`
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    #Passed to KrylovKit
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)
    #Is the Hamiltonian H hermitian? Useful to KrylovKit for exponentiation
    hermitian = get(kwargs,:hermitian,true)
    
    maxdim::Integer = get(kwargs, :maxdim, typemax(Int))
    mindim::Integer = get(kwargs, :mindim, 1)
    cutoff::Real = get(kwargs, :cutoff, 1E-16)
    
    #If progress bar is used
    #pbar = get(kwargs,:progress, true) ? Progress(num_time_steps, desc="Evolving state... ") : nothing
    
   #Imaginary time step
    τ = 1im*dt
    
    N = length(ψ)
    #start with right orthogonalised wave function
    ITensors.orthogonalize!(ψ,1)
    
    #Projected MPO
    #https://docs.juliahub.com/ITensors/P3pqL/0.1.7/ProjMPO.html
    
    PH = ProjMPO(H)
    position!(PH,ψ,1)
    
    
    
    set_nsite!(PH, nsite)
    
    
    maxtruncerr = 0.0
    
    
    #Do the time evolution
    for time_step in 1:num_time_steps
        stime  = @elapsed begin
        # b is the active site  ha gives 1/2 depending on right/left swap
        current_time = dt*time_step
        for (b,ha) in sweepnext(N; ncenter=nsite)
                # Do 'forwards' evolution step
                
                
                set_nsite!(PH, nsite)
                #orthogonalise at position b
                
                position!(PH, ψ, b)
                if nsite == 1
                  #println(" Forward phi1 Selected")
                  phi1 = ψ[b]
                elseif nsite == 2
                  phi1 = ψ[b] * ψ[b + 1]
                end

                
                #orthogonalise at position b
                
                #phi1, info = exponentiate(PH, -τ/2, phi1; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                phi1, info = solver(PH, -τ/2, phi1)
                info.converged==0 && throw("exponentiate did not converge")
                
                spec = nothing
                
                
                if nsite == 1
                  ψ[b] = phi1
                  Sz = measure_Sz(ψ,b) 
                  #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
                  flush(stdout)
                  #println(" Forward phi1 Replaced")
                elseif nsite == 2
                  #Determine sweep direction
                  dir = ha == 1 ? "left" : "right"
                  spec = replacebond!(ψ,b,phi1;mindim,cutoff,normalize=normalize, ortho = dir, kwargs... )
                  maxtruncerr = max(maxtruncerr, spec.truncerr)
                end
                
                if(reverse_step_val == true)
                    if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))


                        b1 = (ha==1 ? b + 1 : b)
                        Δ = (ha==1 ? +1 : -1)

                        if nsite == 2
                            phi0 = ψ[b1]
                        elseif nsite == 1
                            #println(" SVD")
                            uinds = uniqueinds(phi1, ψ[b + Δ])
                            U, S, V = svd(phi1, uinds)
                            ψ[b] = U
                            phi0 = S * V
                            if ha == 1
                              ITensors.setleftlim!(ψ, b)
                            elseif ha == 2
                              ITensors.setrightlim!(ψ, b)
                            end
                        end

                        # evolve with single-site Hamiltonian backward in time.
                        # In the case of imaginary time-evolution this step
                        # is not necessary (see Ref. [1])




                        set_nsite!(PH, nsite - 1)
                        ITensors.position!(PH, ψ, b1)
                        phi0, info = exponentiate(PH, τ/2, phi0; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                        normalize && (phi0 ./= norm(phi0))
                        info.converged==0 && throw("exponentiate did not converge")


                        if nsite == 2
                            ψ[b1] = phi0
                            #print(" Reverse phi  calculated")
                        elseif nsite == 1
                            #println(" Reverse phi  calculated")
                            ψ[b + Δ] = phi0 * ψ[b + Δ]
                            if ha == 1
                              ITensors.setrightlim!(ψ, b + Δ + 1)
                            elseif ha == 2
                              ITensors.setleftlim!(ψ, b + Δ - 1)
                            end
                        end

                        set_nsite!(PH, nsite)



                    end
                    #if ((ha == 1 && (b + nsite - 1 != N)) || (ha == 2 && b != 1))
                end
                #for if(reverse_step_val == true)
        end
        #for(b,ha)
            
            #Sz = measure_Sz(ψ,5) 
            #@printf("Sweep %d,  Sz %f ----------------------------\n", time_step , Sz)
            flush(stdout)

        
        end
        #time elapsed
        
        
       
        
    end
    #External for loop for time_step
    
    ψ=ψ/norm(ψ)
end
#function


tdvp! (generic function with 1 method)

In [68]:
#Number of spin 1/2 sites
N = 10

#fnal time
tf = 2.0
#time step
tau = 0.05


s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N - 1)
    os += 0.5, "S+", j, "S-", j + 1
    os += 0.5, "S-", j, "S+", j + 1
    os += "Sz", j, "Sz", j + 1
end

H = MPO(os, s)
#ψ = productMPS(s, n -> isodd(n) ? "Up" : "Dn")
ψ=randomMPS(s; linkdims=5)
#psi = productMPS(s, n -> isodd(n) ? "Up" : "Dn")


for j=1:N
    println("site : ",j," ",measure_Sz(ψ,j))
end

ψ4 = deepcopy(ψ)
ψ5 = deepcopy(ψ)

site : 1 0.2367518444967298
site : 2 0.08275176239449578
site : 3 0.2059104869685403
site : 4 0.038021509776564544
site : 5 0.16951189208694806
site : 6 0.12788123018316128
site : 7 0.19684781054143535
site : 8 0.012635597866778106
site : 9 0.05089615323377697
site : 10 -0.11185920944766875


MPS
[1] ((dim=2|id=922|"S=1/2,Site,n=1"), (dim=5|id=95|"Link,l=1"))
[2] ((dim=5|id=95|"Link,l=1"), (dim=2|id=308|"S=1/2,Site,n=2"), (dim=5|id=785|"Link,l=2"))
[3] ((dim=5|id=785|"Link,l=2"), (dim=2|id=243|"S=1/2,Site,n=3"), (dim=5|id=492|"Link,l=3"))
[4] ((dim=5|id=492|"Link,l=3"), (dim=2|id=33|"S=1/2,Site,n=4"), (dim=5|id=554|"Link,l=4"))
[5] ((dim=5|id=554|"Link,l=4"), (dim=2|id=188|"S=1/2,Site,n=5"), (dim=5|id=592|"Link,l=5"))
[6] ((dim=5|id=592|"Link,l=5"), (dim=2|id=868|"S=1/2,Site,n=6"), (dim=5|id=595|"Link,l=6"))
[7] ((dim=5|id=595|"Link,l=6"), (dim=2|id=25|"S=1/2,Site,n=7"), (dim=5|id=303|"Link,l=7"))
[8] ((dim=5|id=303|"Link,l=7"), (dim=2|id=289|"S=1/2,Site,n=8"), (dim=4|id=210|"Link,l=8"))
[9] ((dim=4|id=210|"Link,l=8"), (dim=2|id=110|"S=1/2,Site,n=9"), (dim=2|id=329|"Link,l=9"))
[10] ((dim=2|id=329|"Link,l=9"), (dim=2|id=152|"S=1/2,Site,n=10"))


In [69]:
ψ1 = deepcopy(ψ)
#2-site TDVP
tdvp!(ψ1,H,tau,tf,2,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.
for j=1:N
    println("site : ",j," ",measure_Sz(ψ1,j))
end



site : 1 0.12770447306376845
site : 2 0.21019161741908254
site : 3 0.15914259370150205
site : 4 0.10037771017381955
site : 5 0.08248028628260747
site : 6 0.19266048362443683
site : 7 0.09429664665785806
site : 8 0.006161759939219462
site : 9 0.07529687874110505
site : 10 -0.038963372749321934


In [70]:
ψ2 = deepcopy(ψ)

#1-site TDVP
tdvp!(ψ2,H,tau,tf,1,true,exp_tol=1e-15)

#Calculate magnetisation at all sites.

for j=1:N
    println("site : ",j," ",measure_Sz(ψ2,j))
end



site : 1 0.1277520510839415
site : 2 0.20929808151577356
site : 3 0.15791094898215063
site : 4 0.10387127168738519
site : 5 0.0708390696374756
site : 6 0.2663160888517819
site : 7 0.06908137591752601
site : 8 -0.03558553255901245
site : 9 0.07716741339426948
site : 10 -0.03733449502191398


In [71]:
#TEBD
ψ3 = deepcopy(ψ)
cutoff = 1e-15


ttotal = tf


# Make gates (1,2),(2,3),(3,4),...
gates = ITensor[]
for j in 1:(N - 1)
s1 = s[j]
s2 = s[j + 1]
hj =
  op("Sz", s1) * op("Sz", s2) +
  1 / 2 * op("S+", s1) * op("S-", s2) +
  1 / 2 * op("S-", s1) * op("S+", s2)
Gj = exp(-im * tau / 2 * hj)
push!(gates, Gj)
end
# Include gates in reverse order too
# (N,N-1),(N-1,N-2),...
append!(gates, reverse(gates))
c=div(N,2)

for t in 0.0:tau:ttotal
    Sz = expect(ψ3, "Sz"; sites=c)
    #println("$t $Sz")

    t≈ttotal && break

    ψ3 = apply(gates, ψ3; cutoff)
    normalize!(ψ3)
end


#Measure magnetisation at each site
for j=1:N
    println("site : ",j," ",measure_Sz(ψ3,j))
end



site : 1 0.12771333771534496
site : 2 0.21019555250078462
site : 3 0.1591292496272938
site : 4 0.10036877534163866
site : 5 0.08248293940361821
site : 6 0.19268537337812414
site : 7 0.09428973007312696
site : 8 0.006149644215676563
site : 9 0.07528376580162
site : 10 -0.03894928995646885


In [72]:
println(inner(ψ1,ψ2))

0.8510057331882442 - 0.012950845270412972im


In [73]:
println(inner(ψ1,ψ3))

0.9999999582363946 - 4.363410730345851e-6im


In [74]:
println(inner(ψ2,ψ3))

0.8510288640230905 + 0.012940034198475059im


In [55]:
println(ψ3)

MPS
[1] ((dim=2|id=989|"S=1/2,Site,n=1"), (dim=2|id=587|"Link,n=1"))
[2] ((dim=2|id=587|"Link,n=1"), (dim=2|id=750|"S=1/2,Site,n=2"), (dim=4|id=977|"Link,n=1"))
[3] ((dim=2|id=528|"S=1/2,Site,n=3"), (dim=8|id=844|"Link,n=1"), (dim=4|id=977|"Link,n=1"))
[4] ((dim=2|id=923|"S=1/2,Site,n=4"), (dim=16|id=715|"Link,n=1"), (dim=8|id=844|"Link,n=1"))
[5] ((dim=2|id=707|"S=1/2,Site,n=5"), (dim=31|id=329|"Link,n=1"), (dim=16|id=715|"Link,n=1"))
[6] ((dim=2|id=383|"S=1/2,Site,n=6"), (dim=16|id=683|"Link,n=1"), (dim=31|id=329|"Link,n=1"))
[7] ((dim=2|id=571|"S=1/2,Site,n=7"), (dim=8|id=153|"Link,n=1"), (dim=16|id=683|"Link,n=1"))
[8] ((dim=2|id=210|"S=1/2,Site,n=8"), (dim=4|id=946|"Link,n=1"), (dim=8|id=153|"Link,n=1"))
[9] ((dim=2|id=912|"S=1/2,Site,n=9"), (dim=2|id=506|"Link,n=1"), (dim=4|id=946|"Link,n=1"))
[10] ((dim=2|id=462|"S=1/2,Site,n=10"), (dim=2|id=506|"Link,n=1"))

